### open hotel data

In [53]:
%%time
# to get a cell executation time
# put %%time at the very start of the cell
import pandas as pd
import jieba
import time
#Get Jieba  Dictionary 
jieba.set_dictionary('./jieba_dict/dict.txt.big')
df = pd.read_csv('D:\\your data path\\your data.csv')
print('\ndata qty= ',len(df))
df.head(5)



data qty=  1000
Wall time: 57 ms


,bnb_id,bnb_name,bnb_url,star,price,address,origin_feature,score,image_url,city,x,y
0,10000001,台北109青旅,https://www.booking.com/hotel/tw/109-hostel-ta...,3,1984,台北中正區博愛路36號3樓,['\n\n\n衛浴\n\n\n\n\n\n\n\n\n\n衛生紙\n\n\n\n\n\n\...,9.0,['https://cf.bstatic.com/xdata/images/xphoto/s...,台北市,NaN,NaN
1,10000002,YanjiEarthlyHome4B2b延吉大地之家4房2衛,https://www.booking.com/hotel/tw/2-min-to-mrt-...,3,2977,台北大安區Yanji Street No. 128,"['\n\n\n停車場\n\n\n\n無停車設施。\n\n', '\n\n\n網路\n\n\...",0.0,[],台北市,NaN,NaN
2,10000003,溫馨家庭劇院3房2衛,https://www.booking.com/hotel/tw/3b2b-cinema-s...,3,4868,台北大安區安和路2段32巷27號,['\n\n\n停車場\n\n\n\n需預約：住宿場所設有私人停車設施（（免費））。 \n\...,0.0,[],台北市,NaN,NaN
3,10000004,慶爾喜無人自動入住旅店,https://www.booking.com/hotel/tw/406inn.zh-tw....,2,1188,"台北萬華區區長沙街二段11號2 樓,",['\n\n\n衛浴\n\n\n\n\n\n\n\n\n\n毛巾\n\n\n\n\n\n\n...,6.8,['https://cf.bstatic.com/xdata/images/hotel/ma...,台北市,NaN,NaN
4,10000005,久居棧旅店,https://www.booking.com/hotel/tw/9inehotel.zh-...,3,937,台北大同區延平北路二段255號2樓,['\n\n\n衛浴\n\n\n\n\n\n\n\n\n\n衛生紙\n\n\n\n\n\n\...,8.3,['https://cf.bstatic.com/xdata/images/hotel/ma...,台北市,NaN,NaN


In [54]:
%%time
# to get a cell executation time
# put %%time at the very start of the cell
#Import Jieba Module
import jieba.posseg as pseg
# Combine the ROW
df_feature_ori = df[['bnb_id','bnb_name','star','score','price','origin_feature']]

df_feature = df_feature_ori.copy()

# remove the next Line
df_feature['origin_feature'].replace(r'\s+|\\n', ' ', regex=True, inplace=True) 

print('')
df_feature['bnb_name'].head(5)



Wall time: 147 ms


0                           台北109青旅
1    YanjiEarthlyHome4B2b延吉大地之家4房2衛
2                        溫馨家庭劇院3房2衛
3                       慶爾喜無人自動入住旅店
4                             久居棧旅店
Name: bnb_name, dtype: object

In [55]:
%%time
# to get a cell executation time
# put %%time at the very start of the cell
df_feature.isna().any()

Wall time: 2 ms


bnb_id            False
bnb_name          False
star              False
score             False
price             False
origin_feature    False
dtype: bool

In [56]:

# 清理空值並檢查空值
df_feature=df_feature.dropna(axis=0)
df_feature.isna().any()

bnb_id            False
bnb_name          False
star              False
score             False
price             False
origin_feature    False
dtype: bool

In [57]:
# 清理重複
feature_duplicates = df_feature.duplicated()
if feature_duplicates.sum() > 0:
    print('> {} duplicates'.format(feature_duplicates.sum()))
    df_feature = df_feature[~feature_duplicates]

print('> {} duplicates'.format(df_feature.duplicated().sum()))

> 0 duplicates


In [58]:
print('Rating Dataframe shape : ', df_feature.shape)

Rating Dataframe shape :  (1000, 6)


In [59]:
df_feature.columns

Index(['bnb_id', 'bnb_name', 'star', 'score', 'price', 'origin_feature'], dtype='object')

In [60]:
# 給user_id
import statsmodels.api as sm
df_feature['hotel_id'] = pd.Categorical(df_feature.bnb_name).codes   
df_feature.head()

,bnb_id,bnb_name,star,score,price,origin_feature,hotel_id
0,10000001,台北109青旅,3,9.0,1984,[' 衛浴 衛生紙 額外衛浴 額外...,221
1,10000002,YanjiEarthlyHome4B2b延吉大地之家4房2衛,3,0.0,2977,"[' 停車場 無停車設施。 ', ' 網路 住宿方于全館提供WiFi（...",82
2,10000003,溫馨家庭劇院3房2衛,3,0.0,4868,[' 停車場 需預約：住宿場所設有私人停車設施（（免費））。 無...,641
3,10000004,慶爾喜無人自動入住旅店,2,6.8,1188,[' 衛浴 毛巾 拖鞋 免費盥洗用...,459
4,10000005,久居棧旅店,3,8.3,937,[' 衛浴 衛生紙 毛巾 拖鞋 ...,102


In [61]:

hotel_features_df = df_feature.pivot_table(index = 'bnb_name',columns ='hotel_id' ,values = 'score')
hotel_features_df.head(20)

hotel_id,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
bnb_name,,,,,,,,,,,,,,,,,,,,,
106民宿,8.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24度42分民宿,NaN,9.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26旅店,NaN,NaN,8.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3Kiss民宿,NaN,NaN,NaN,9.7,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
512木屋民宿,NaN,NaN,NaN,NaN,8.7,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58度溫泉湯苑會館,NaN,NaN,NaN,NaN,NaN,7.9,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5963右轉民宿,NaN,NaN,NaN,NaN,NaN,NaN,9.4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7號童趣聯盟,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.7,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8688民宿,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.8,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
missing_value = hotel_features_df.isna().sum()
missing_value

hotel_id
0      999
1      999
2      999
3      999
4      999
      ... 
995    999
996    999
997    999
998    999
999    999
Length: 1000, dtype: int64

In [63]:
rows = hotel_features_df.shape[0]
cols = hotel_features_df.shape[1]

count_empty_or_zero_cells = hotel_features_df.isna().sum().sum()
total_elements = rows * cols

sparsity_of_matrix = count_empty_or_zero_cells/total_elements

print('Total Empty cells are : ', count_empty_or_zero_cells)
print('Total cells in Matrix are : ', total_elements)
print('Sparsity of Matrix are : ', sparsity_of_matrix)

Total Empty cells are :  999000
Total cells in Matrix are :  1000000
Sparsity of Matrix are :  0.999


In [64]:
# 空值用 0 取代
hotel_features_df.fillna(0, inplace = True)

In [65]:
hotel_features_df.values

array([[8.9, 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 9.4, 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 8.6, ..., 0. , 0. , 0. ],
       ...,
       [0. , 0. , 0. , ..., 8.1, 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 8.2, 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 8.1]])

In [66]:
from scipy.sparse import csr_matrix
hotel_features_df_matrix = csr_matrix(hotel_features_df.values)

In [67]:

# 建立歐式相似模型 (K-近鄰演算法)
from sklearn.neighbors import NearestNeighbors
nearest_neighbor_model = NearestNeighbors(metric = 'euclidean', algorithm = 'brute')
nearest_neighbor_model.fit(hotel_features_df_matrix)


NearestNeighbors(algorithm='brute', metric='euclidean')

In [68]:
import numpy as np         
# 隨機選擇
total_no_of_hotels = hotel_features_df.shape[0]
print('Total hotels in our pivot table : ', total_no_of_hotels)
random_hotels_index = np.random.choice(total_no_of_hotels)
print('Random hotels Index : ', random_hotels_index)


Total hotels in our pivot table :  1000
Random hotels Index :  714


In [69]:
hotel_features_df.iloc[random_hotels_index]

hotel_id
0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
995    0.0
996    0.0
997    0.0
998    0.0
999    0.0
Name: 福格大飯店, Length: 1000, dtype: float64

In [70]:
# 向量顯示 random hotels
one_dimensional_representation_of_hotels_vector = hotel_features_df.iloc[random_hotels_index].values.reshape(1, -1)
one_dimensional_representation_of_hotels_vector

array([[0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. 

In [71]:
# 相似hotel 與 hotel 的歐式距離指數
distances, indices = nearest_neighbor_model.kneighbors(one_dimensional_representation_of_hotels_vector, n_neighbors = 6)
print('Distance :', distances)
print('Indices :', indices)

Distance : [[0.  7.7 7.7 7.7 7.7 7.7]]
Indices : [[714  53 481  51 594 495]]


In [72]:
indices = indices.flatten()
distances = distances.flatten()
for i in range(0, len(indices)):
    ## same hotels
    if i == 0:
        print('Recommendations for {0}:\n'.format(hotel_features_df.index[random_hotels_index]))
    else:
        ## similar hotels
        print('{0}: {1}, with distance of {2}'.format(i, hotel_features_df.index[indices[i]], distances[i]))

Recommendations for 福格大飯店:

1: MRWATER, with distance of 7.7
2: 新三12公寓, with distance of 7.7
3: LoftSunnyHome5PPLStaynear101, with distance of 7.7
4: 河岸好日子3館, with distance of 7.7
5: 旅夜書懷年輪親子民宿, with distance of 7.7


In [73]:

# 找出全部hotels的相似資料集 
hotel_dict = {}
for hotel_index in range(hotel_features_df.shape[0]):
    one_dimensional_representation_of_hotel_vector = hotel_features_df.iloc[hotel_index].values.reshape(1, -1)
    distances, indices = nearest_neighbor_model.kneighbors(one_dimensional_representation_of_hotel_vector, n_neighbors = 4)
    indices = indices.flatten()
    distances = distances.flatten()
    similar_hotel = []
    
    for i in range(0, len(indices)):
        ## same hotels
        if i == 0:
            original_hotel = hotel_features_df.index[hotel_index]
        else:
            ## similar hotels
            similar_hotel.append(hotel_features_df.index[indices[i]])
        hotel_dict[original_hotel] = similar_hotel

In [74]:
# 秀出前hotel相似
recommended_hotel_df = pd.DataFrame(hotel_dict)
recommended_hotel_df = recommended_hotel_df.T
recommended_hotel_df.columns = ['1st_Similar hotel ', '2nd_Similar hotel ', '3rd_Similar hotel ']
recommended_hotel_df.head(20)

,1st_Similar hotel,2nd_Similar hotel,3rd_Similar hotel
106民宿,MRWATER,M2Villa泳池親子包棟民宿,LoftSunnyHome5PPLStaynear101
24度42分民宿,MRWATER,LoftSunnyHome5PPLStaynear101,LioanGuvaObelay
26旅店,宜蘭樂見冬山民宿,宜蘭頭城空空如也KongKongBB,萬九126公寓
3Kiss民宿,宜蘭頭城空空如也KongKongBB,萬九126公寓,宜蘭樂見冬山民宿
512木屋民宿,松菸553公寓,萬九126公寓,宜蘭頭城空空如也KongKongBB
58度溫泉湯苑會館,MRWATER,LoftSunnyHome5PPLStaynear101,LioanGuvaObelay
5963右轉民宿,宜蘭頭城空空如也KongKongBB,萬九126公寓,松菸553公寓
7號童趣聯盟,LoftSunnyHome5PPLStaynear101,LioanGuvaObelay,錢三
8688民宿,宜蘭頭城空空如也KongKongBB,萬九126公寓,宜蘭樂見冬山民宿
AHOME,宜蘭頭城空空如也KongKongBB,田宿,萬九126公寓


In [75]:
# hotel recommandendation
hotel_name = "宜蘭樂見冬山民宿"
recommended_hotel_df.head(20)
# result = recommended_hotel_df.loc[hotel_name, : ]
# print('I have recommended_hotel_df  for you: \n')
# for hotels in list(result.values):
#       print(hotels)

,1st_Similar hotel,2nd_Similar hotel,3rd_Similar hotel
106民宿,MRWATER,M2Villa泳池親子包棟民宿,LoftSunnyHome5PPLStaynear101
24度42分民宿,MRWATER,LoftSunnyHome5PPLStaynear101,LioanGuvaObelay
26旅店,宜蘭樂見冬山民宿,宜蘭頭城空空如也KongKongBB,萬九126公寓
3Kiss民宿,宜蘭頭城空空如也KongKongBB,萬九126公寓,宜蘭樂見冬山民宿
512木屋民宿,松菸553公寓,萬九126公寓,宜蘭頭城空空如也KongKongBB
58度溫泉湯苑會館,MRWATER,LoftSunnyHome5PPLStaynear101,LioanGuvaObelay
5963右轉民宿,宜蘭頭城空空如也KongKongBB,萬九126公寓,松菸553公寓
7號童趣聯盟,LoftSunnyHome5PPLStaynear101,LioanGuvaObelay,錢三
8688民宿,宜蘭頭城空空如也KongKongBB,萬九126公寓,宜蘭樂見冬山民宿
AHOME,宜蘭頭城空空如也KongKongBB,田宿,萬九126公寓
